In [ ]:
# Leave for imports

Run the following !pip3 cells to install scikit-plot and tensorflow which are needed for this sandbox.

In [ ]:
!pip3 install scikit-plot

In [ ]:
!pip3 install tensorflow

Welcome to automating neural networks with Arctic Fox! Our focus of these automations is to make developing AI / ML solutions more akin to how we think about AI / ML - get data, build the network, train the model, and view the results. 

# Automations with Nueral Network Classifier


We all love AI! Well, most of us... And it's amazing how much easier it is to develop AI solutions today than in the past. But, what we found, is that a lot of times when we take on a new project, try to recycle an old notebook, etc. there's always more edits / refactoring than we thought. So, we figured it would be easier if a lot of it was automated!

To do this, we created 4 automations - Data, NeuralNetwork (NN), Train, and Visualize. We placed them each below, with some description, and pre-populated some parameters that worked for us. Play around with it and enjoy!

##### Note
The examples in codespace use simple csvs and neural networks due to processing power constraints. Automations exist for convolutional neural networks as well. However, for non-gpu environments, training is too slow to show meaningful examples. 

### Data, Load CSV

The first automation will load in the data. In this case, our source is a csv. Other times, the source could be a library in an imported source, a collection of images, etc. Additionally, we can tell it what to predict - we could tell it the name of a column or the column index. Our example uses the popular wine dataset, and since there are no headers, we give it the column index. 

Additionally, by providing column index 0, Arctic Fox will look at the data and determine that the desired model is a classifier. 

In [ ]:
#[Data wine.csv --predict 0]

### Build Neural Network

Next, we use the NeuralNetwork automation, or NN for short. We have provided some example parameters, but if you didn't have these, you could always just use #[NN --help] to see the available options. The start and end specify the number of nodes in the start and end (second to last) layers. DenseLayers specifies how many layers should be in the model. 

Play around with the parameters to see how the model changes.  

In [ ]:
#[NN --denseLayers 20 --denseStart 96 --denseEnd 32 --dropOutRatio 1:5]

### Train Model


Pretty much the only thing to specify when training is how long to train, or the number of epochs. You can always re-run the training, or fit, cell to do more training. Train, retrain, keep training, that's all pretty normal. 

In [ ]:
#[Train --epochs 300]

### Visualize

Finally, it's always import to visualize a model's performance. Seeing the training loss and accuracy evolve over time let's you get a feel fore how the model trained. We think for a classifier that the confusion matrix is one of the most telling visuals. 

In [ ]:
#[Visualize --loss --accuracy --confusionMatrix]

That's a wrap on the basics of automating neural networks. The next sandbox makes a small change, and instantly your notebook is setup for regression. Check it out!